In [1]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook

%matplotlib inline
%load_ext autoreload
%autoreload 2

from comet_ml import Experiment

##### TODO
- Test the ``chess`` library
    - Convert to numpy array
    - Gif 
- Use pre-existing GameAI
- Create GameAI with minimax techniques
- Evaluation function for each move
- Move recommendation "à la" AlphaGo, MCTS ?
- Notebook interface to play
- Adaptive GameAI



# Test of the ``chess`` library
https://python-chess.readthedocs.io/en/latest/



# Playground

In [2]:
import sys
sys.path.append("../")

from beth.game import Game
from beth.players.random_player import RandomPlayer
from beth.players.human_player import HumanPlayer
from beth.players.sequence import SequenceGame,SequencePlayer

In [3]:
# black = RandomPlayer()
# white = RandomPlayer()

white = HumanPlayer()
black = HumanPlayer()

game = Game(white,black)

In [131]:
game.reset_game()
game.run()

Output()

WHITE move: e4
BLACK move: e5
WHITE move: Nf3
BLACK move: Nf6
WHITE move: exit


In [5]:
game.board.is_seventyfive_moves()

True

# Game exploration

In [3]:
games = pd.read_csv("../data/raw/kaggle/games.csv")

In [4]:
games.shape

(20058, 16)

In [5]:
"start " + games["moves"].head()

0    start d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ N...
1    start d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Q...
2    start e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 a...
3    start d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2...
4    start e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be...
Name: moves, dtype: object

In [6]:
game = games.iloc[2]

seq_str = game["moves"]
winner = game["winner"]
victory_status = game["victory_status"]

In [7]:
seq = SequenceGame(seq_str,0.1,victory_status,winner)
game = Game(seq.white,seq.black)
seq_str

'e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc6 bxc6 Ra6 Nc4 a4 c3 a3 Nxa3 Rxa3 Rxa3 c4 dxc4 d5 cxd5 Qxd5 exd5 Be6 Ra8+ Ke7 Bc5+ Kf6 Bxf8 Kg6 Bxg7 Kxg7 dxe6 Kh6 exf7 Nf6 Rxh8 Nh5 Bxh5 Kg5 Rxh7 Kf5 Qf3+ Ke6 Bg4+ Kd6 Rh6+ Kc5 Qe3+ Kb5 c4+ Kb4 Qc3+ Ka4 Bd1#'

In [8]:
seq.victory_status

'mate'

In [9]:
game.run()

Output()

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

# Experiment 1 using LSTMs
https://www.kdnuggets.com/2020/07/pytorch-lstm-text-generation-tutorial.html

In [3]:
from beth.models.dataset import Dataset
from beth.utils import make_experiment
from beth.models.lstm import LSTMModel

## Init classes

In [4]:
games = pd.read_csv("../data/raw/kaggle/games.csv")
moves = "start " + games["moves"]
moves.head()

0    start d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ N...
1    start d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Q...
2    start e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 a...
3    start d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2...
4    start e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be...
Name: moves, dtype: object

In [5]:
dataset = Dataset(moves)

In [6]:
len(dataset.uniq_words)

4448

## Train loop

In [37]:
experiment = make_experiment("../.env","Simple LSTM",["NLP","LSTM"])

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/theolvs/beth/2f28f4cbecb646179c6a3d4a0ca2b444



In [38]:
params = {
    "sequence_length":10,
    "batch_size":32,
    "max_epochs":1,
    "lr":0.01,
    "lstm_size":128,
    "embedding_dim":128,
    "num_layers":3,
}

In [39]:
model = LSTMModel(dataset,**params)

In [ ]:
experiment.log_parameters(params)
model.fit(**params)
# experiment.end()

In [32]:
experiment.log_text("With more embeddings and layers, it just take too much time to learn anything")

{'web': 'https://www.comet.ml/api/asset/download?assetId=e21af61a5b7a4c1fbe430b99569c8d9e&experimentKey=dd08972e62dc4c19b9ab53d1209af2d6',
 'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=e21af61a5b7a4c1fbe430b99569c8d9e&experimentKey=dd08972e62dc4c19b9ab53d1209af2d6',
 'assetId': 'e21af61a5b7a4c1fbe430b99569c8d9e'}

In [35]:
import torch

torch.save(model.state_dict(),"experiment.pth")

In [36]:
!start .

In [33]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/theolvs/beth/dd08972e62dc4c19b9ab53d1209af2d6
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [727] : (5.02025842666626, 8.408011436462402)
COMET INFO:   Others:
COMET INFO:     Name : Simple LSTM
COMET INFO:   Parameters:
COMET INFO:     batch_size      : 32
COMET INFO:     embedding_dim   : 256
COMET INFO:     lr              : 0.001
COMET INFO:     lstm_size       : 256
COMET INFO:     max_epochs      : 1
COMET INFO:     num_layers      : 5
COMET INFO:     sequence_length : 10
COMET INFO:   Uploads:
COMET INFO:     code                     : 1 (2 KB)
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (1

In [26]:
p = model.predict("start e4",10,as_proba = True)

In [27]:
p[0].sort_values(ascending = False).head(10)

dxe4     0.081799
fxe4     0.044852
Nxe4     0.036191
Bxe4     0.028277
start    0.020529
Qxe4     0.020383
Rxe4     0.017424
f3       0.013124
f5       0.009175
Nd4      0.008905
Name: 0, dtype: float64

In [92]:
# black = RandomPlayer()
# white = RandomPlayer()

white = HumanPlayer()
black = HumanPlayer()

game = Game(white,black)

In [93]:
game.run()

Output()

WHITE move: a4
BLACK move: e5
WHITE move: Nf3
BLACK move: Nc6
WHITE move: Bc4
